![header](../figures/logos_partenaires._cmems_se.jpg)

# Access data

<div style="text-align: right"><i> 2023-06-13 ACCESS_DATA </i></div>

***
**Authors:**  CLS & Datlas <br>
**Copyright:** 2023 CLS & Datlas <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h1><center>Access the data challenge necessary data</center></h1>
<h5> The notebook aims to illustrate how to download and read the various necessary data to reconstruct maps (experiment), to evaluate the maps performances (evaluation) and to compare with other reconstructions (comparison). </h5> 
</div>


<div class="alert alert-danger" role="alert">

<h3><center> Warning: To run this notebook you will need a free disk space of approximately 33Go.</center></h3>
 
    

***
**General Note 1**: Execute each cell through the <button class="btn btn-default btn-xs"><i class="icon-play fa fa-play"></i></button> button from the top MENU (or keyboard shortcut `Shift` + `Enter`).<br>
<br>
**General Note 2**: If, for any reason, the kernel is not working anymore, in the top MENU, click on the <button class="btn btn-default btn-xs"><i class="fa fa-repeat icon-repeat"></i></button> button. Then, in the top MENU, click on "Cell" and select "Run All Above Selected Cell".<br>
***


<div class="alert alert-danger" role="alert">

<h3>Learning outcomes</h3>

At the end of this notebook you will know:
<ul>
  <li>How you can download and read all the necessary data for this data challenge.</li>
</ul>
    
</div>

In [1]:
from glob import glob
import numpy as np
import os
import sys
sys.path.append('../')
import xarray as xr
from src.mod_read import *

<div class="alert alert-info" role="alert">

<h2>0. Parameters</h2>

</div>

In [2]:
time_min = '2019-01-01'                                        # time min for analysis
time_max = '2019-12-31'                                        # time max for analysis
output_dir = '../results'                                      # output directory path
os.system(f'mkdir -p {output_dir}')
stat_output_filename = f'{output_dir}/stat_sla_duacs_geos.nc'  # output statistical analysis filename
lambda_min = 65.                                               # minimun spatial scale in kilometer to consider on the filtered signal
lambda_max = 500.                                              # maximum spatial scale in kilometer to consider on the filtered signal
psd_output_filename = f'{output_dir}/psd_sla_duacs_geos.nc'    # output spectral analysis filename
segment_lenght = 1000.                                         # spectral parameer: along-track segment lenght in kilometer to consider in the spectral analysis

<div class="alert alert-info" role="alert">

<h2>1. Data for experiment</h2>

</div>

## Download the experiment alongtrack nadir data

In [6]:
if os.path.isdir('../data/alongtrack'):
    print('Alongtrack data already exist locally in ../data/')
else: 
    print('Downloading alongtrack data')
    
    # Download the alongtrack repository
    !wget https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/MEOM/OCEAN_DATA_CHALLENGES/2023a_SSH_mapping_OSE/alongtrack.tar.gz
    
    # Uncompress in ../data
    !tar -xvf alongtrack.tar.gz --directory ../data
    
    # Remove compressed file
    !rm -f alongtrack.tar.gz
    

--2025-03-07 13:19:57--  https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/MEOM/OCEAN_DATA_CHALLENGES/2023a_SSH_mapping_OSE/alongtrack.tar.gz
Résolution de ige-meom-opendap.univ-grenoble-alpes.fr (ige-meom-opendap.univ-grenoble-alpes.fr)… 129.88.191.39
Connexion à ige-meom-opendap.univ-grenoble-alpes.fr (ige-meom-opendap.univ-grenoble-alpes.fr)|129.88.191.39|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 200
Taille : 2429204480 (2,3G) [application/octet-stream]
Enregistre : ‘alongtrack.tar.gz’

alongtrack.tar.gz   100%[===================>]   2,26G  6,75MB/s    ds 6m 23s  

2025-03-07 13:26:20 (6,05 MB/s) - ‘alongtrack.tar.gz’ enregistré [2429204480/2429204480]

alongtrack/
alongtrack/h2ag/
alongtrack/h2ag/2018/
alongtrack/h2ag/2018/dt_global_h2ag_phy_l3_20181129_20210603.nc
alongtrack/h2ag/2018/dt_global_h2ag_phy_l3_20181102_20210603.nc
alongtrack/h2ag/2018/dt_global_h2ag_phy_l3_20180504_20210603.nc
alongtrack/h2ag/2018/dt_gl

<div class="alert alert-info" role="alert">

<h2>2. Data for evaluation</h2>

</div>

## Download the independant alongtrack nadir data (Saral/AltiKa)

In [ ]:
if os.path.isdir('../data/independent_alongtrack'):
    print('Independant alongtrack data already exist locally in ../data/')
else: 
    print('Downloading independant alongtrack data')
    
    # Download the alongtrack repository
    !wget https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/MEOM/OCEAN_DATA_CHALLENGES/2023a_SSH_mapping_OSE/independent_alongtrack.tar.gz
    
    # Uncompress in ../data
    !tar -xvf independent_alongtrack.tar.gz --directory ../data
    
    # Remove compressed file
    !rm -f independent_alongtrack.tar.gz 
    

## Read the independant alongtrack nadir data (Saral/AltiKa)

In [3]:
%%time
list_of_file = sorted(glob('../data/independent_alongtrack/alg/2019/*.nc'))
ds_alg = xr.open_mfdataset(list_of_file, combine='nested', concat_dim='time')
ds_alg = ds_alg.where((ds_alg.time >= np.datetime64(time_min)) & (ds_alg.time <=  np.datetime64(time_max)), drop=True)
ds_alg = ds_alg.sortby('time')
ds_alg

CPU times: user 12min 17s, sys: 21.4 s, total: 12min 38s
Wall time: 12min 48s


<xarray.Dataset>
Dimensions:         (time: 14437696)
Coordinates:
  * time            (time) datetime64[ns] 2019-01-01T00:04:07.003014144 ... 2...
    longitude       (time) float64 dask.array<chunksize=(39664,), meta=np.ndarray>
    latitude        (time) float64 dask.array<chunksize=(39664,), meta=np.ndarray>
Data variables:
    cycle           (time) float32 dask.array<chunksize=(39664,), meta=np.ndarray>
    track           (time) float32 dask.array<chunksize=(39664,), meta=np.ndarray>
    sla_unfiltered  (time) float32 dask.array<chunksize=(39664,), meta=np.ndarray>
    sla_filtered    (time) float32 dask.array<chunksize=(39664,), meta=np.ndarray>
    dac             (time) float32 dask.array<chunksize=(39664,), meta=np.ndarray>
    ocean_tide      (time) float32 dask.array<chunksize=(39664,), meta=np.ndarray>
    internal_tide   (time) float32 dask.array<chunksize=(39664,), meta=np.ndarray>
    lwe             (time) float32 dask.array<chunksize=(39664,), meta=np.ndarray>
    mdt             (time) float32 dask.array<chunksize=(39664,), meta=np.ndarray>
    tpa_correction  (time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Attributes: (12/44)
    Conventions:                     CF-1.6
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Swath
    comment:                         Sea surface height measured by altimeter...
    contact:                         servicedesk.cmems@mercator-ocean.eu
    creator_email:                   servicedesk.cmems@mercator-ocean.eu
    ...                              ...
    summary:                         SSALTO/DUACS Delayed-Time Level-3 sea su...
    time_coverage_duration:          P24H11M29.716548S
    time_coverage_end:               2019-01-01T23:36:52Z
    time_coverage_resolution:        P1S
    time_coverage_start:             2018-12-31T23:25:22Z
    title:                           DT Altika Drifting Phase Global Ocean Al...

## Download the independant drifter data

In [5]:
%%time

if os.path.isdir('../data_dev/independent_drifters0'):
    print('Independant drifter data already exist locally in ../data/')
else: 
    print('Downloading independent drifter data')
    
    path_catalog = "https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/catalog/meomopendap/extract/MEOM/OCEAN_DATA_CHALLENGES/2023a_SSH_mapping_OSE/independent_drifters/catalog.html" 
    path_data =    "https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/MEOM/OCEAN_DATA_CHALLENGES/2023a_SSH_mapping_OSE/independent_drifters/"


    list_of_files = retrieve_list_of_files_from_url(path_catalog, path_data,None,False)

    !mkdir ../data_dev/independant_drifters0

    i = 0
    for ifile in sorted(list_of_files)[:]:
        print(ifile) 
        os.system("wget -P ../data_dev/independant_drifters0/ i {}".format(ifile))
    

mkdir: impossible de créer le répertoire «../data_dev/independant_drifters0»: Aucun fichier ou dossier de ce nom
https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/MEOM/OCEAN_DATA_CHALLENGES/2023a_SSH_mapping_OSE/independent_drifters/ Documentation


--2025-03-07 14:37:16--  http://i/
Résolution de i (i)… échec : Échec temporaire dans la résolution du nom.
wget : impossible de résoudre l’adresse de l’hôte ‘i’
--2025-03-07 14:37:16--  https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/MEOM/OCEAN_DATA_CHALLENGES/2023a_SSH_mapping_OSE/independent_drifters/
Résolution de ige-meom-opendap.univ-grenoble-alpes.fr (ige-meom-opendap.univ-grenoble-alpes.fr)… 129.88.191.39
Connexion à ige-meom-opendap.univ-grenoble-alpes.fr (ige-meom-opendap.univ-grenoble-alpes.fr)|129.88.191.39|:443… connecté.
requête HTTP transmise, en attente de la réponse… 400 400
2025-03-07 14:37:16 erreur 400 : 400.

--2025-03-07 14:37:16--  http://documentation/
Résolution de documentation (documentation)… échec : Échec temporaire dans la résolution du nom.
wget : impossible de résoudre l’adresse de l’hôte ‘documentation’
--2025-03-07 14:37:16--  http://i/
Résolution de i (i)… échec : Échec temporaire dans la résolution du nom.
wget 

https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/MEOM/OCEAN_DATA_CHALLENGES/2023a_SSH_mapping_OSE/independent_drifters/ Info 


requête HTTP transmise, en attente de la réponse… 400 400
2025-03-07 14:37:16 erreur 400 : 400.

--2025-03-07 14:37:16--  http://info/
Résolution de info (info)… échec : Échec temporaire dans la résolution du nom.
wget : impossible de résoudre l’adresse de l’hôte ‘info’
--2025-03-07 14:37:16--  http://i/
Résolution de i (i)… échec : Échec temporaire dans la résolution du nom.
wget : impossible de résoudre l’adresse de l’hôte ‘i’
--2025-03-07 14:37:16--  https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/MEOM/OCEAN_DATA_CHALLENGES/2023a_SSH_mapping_OSE/independent_drifters/.ipynb_checkpoints/
Résolution de ige-meom-opendap.univ-grenoble-alpes.fr (ige-meom-opendap.univ-grenoble-alpes.fr)… 129.88.191.39
Connexion à ige-meom-opendap.univ-grenoble-alpes.fr (ige-meom-opendap.univ-grenoble-alpes.fr)|129.88.191.39|:443… connecté.


https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/MEOM/OCEAN_DATA_CHALLENGES/2023a_SSH_mapping_OSE/independent_drifters/.ipynb_checkpoints/


...

https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/MEOM/OCEAN_DATA_CHALLENGES/2023a_SSH_mapping_OSE/independent_drifters/uv_drifters_20191230.nc


requête HTTP transmise, en attente de la réponse… 200 200
Taille : 234424 (229K) [application/x-netcdf]
Enregistre : ‘../data_dev/independant_drifters0/uv_drifters_20191230.nc’

     0K .......... .......... .......... .......... .......... 21% 1022K 0s
    50K .......... .......... .......... .......... .......... 43% 15,6M 0s
   100K .......... .......... .......... .......... .......... 65% 2,05M 0s
   150K .......... .......... .......... .......... .......... 87% 1,90M 0s
   200K .......... .......... ........                        100%  114M=0,1s

2025-03-07 14:42:57 (2,20 MB/s) - ‘../data_dev/independant_drifters0/uv_drifters_20191230.nc’ enregistré [234424/234424]

Terminé — 2025-03-07 14:42:57 —
Temps total effectif : 0,4s
Téléchargés : 1 fichiers, 229K en 0,1s (2,20 MB/s)
--2025-03-07 14:42:57--  http://i/
Résolution de i (i)… échec : Échec temporaire dans la résolution du nom.
wget : impossible de résoudre l’adresse de l’hôte ‘i’
--2025-03-07 14:42:57--  https://ige-meom-op

https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/MEOM/OCEAN_DATA_CHALLENGES/2023a_SSH_mapping_OSE/independent_drifters/uv_drifters_20191231.nc


Résolution de ige-meom-opendap.univ-grenoble-alpes.fr (ige-meom-opendap.univ-grenoble-alpes.fr)… 129.88.191.39
Connexion à ige-meom-opendap.univ-grenoble-alpes.fr (ige-meom-opendap.univ-grenoble-alpes.fr)|129.88.191.39|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 200
Taille : 73624 (72K) [application/x-netcdf]
Enregistre : ‘../data_dev/independant_drifters0/uv_drifters_20191231.nc’

     0K .......... .......... .......... .......... .......... 69% 1008K 0s
    50K .......... .......... .                               100% 82,3M=0,05s

2025-03-07 14:42:58 (1,41 MB/s) - ‘../data_dev/independant_drifters0/uv_drifters_20191231.nc’ enregistré [73624/73624]

Terminé — 2025-03-07 14:42:58 —
Temps total effectif : 0,3s
Téléchargés : 1 fichiers, 72K en 0,05s (1,41 MB/s)


CPU times: user 14.1 s, sys: 3.82 s, total: 17.9 s
Wall time: 5min 45s


## Read the independant drifter data  

In [8]:
filenames_drifters = sorted(glob('../data_dev/independant_drifters0/uv_drifters_*.nc'))
ds_drifter = xr.open_mfdataset(filenames_drifters, combine='nested', concat_dim='time')
ds_drifter = ds_drifter.where((ds_drifter.time >= np.datetime64(time_min)) & (ds_drifter.time <=  np.datetime64(time_max)), drop=True)
ds_drifter

<xarray.Dataset>
Dimensions:    (time: 2156405)
Coordinates:
  * time       (time) datetime64[ns] 2019-01-01 ... 2019-12-31
    latitude   (time) float32 dask.array<chunksize=(5559,), meta=np.ndarray>
    longitude  (time) float32 dask.array<chunksize=(5559,), meta=np.ndarray>
Data variables:
    EWCT       (time) float32 dask.array<chunksize=(5559,), meta=np.ndarray>
    NSCT       (time) float32 dask.array<chunksize=(5559,), meta=np.ndarray>
    sensor_id  (time) float64 dask.array<chunksize=(5559,), meta=np.ndarray>
Attributes: (12/46)
    data_type:                   OceanSITES trajectory data
    format_version:              2.0
    platform_code:               116275
    date_update:                 2020-10-13T12:17:40Z
    institution:                 AOML
    institution_edmo_code:       1799
    ...                          ...
    deployment_lat:              -58.44
    last_longitude_observation:  82.75
    last_latitude_observation:   -18.49
    date_drog_lost:              2017-01-21T03:37:00Z
    death_type:                  stop transmitting
    last_date_observation:       2019-01-16T01:51:00Z

## Download auxiliary data for diagnostics

In [9]:
if os.path.isdir('../data/sad'):
    print('sad data already exist locally in ../data/')
else: 
    print('Downloading sad data')
    
    # Download the alongtrack repository
    !wget https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/MEOM/OCEAN_DATA_CHALLENGES/2023a_SSH_mapping_OSE/sad.tar.gz
    
    # Uncompress in ../data
    !tar -xvf sad.tar.gz --directory ../data
    
    # Remove compressed file
    !rm -f sad.tar.gz 
    

--2025-03-07 14:45:44--  https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/MEOM/OCEAN_DATA_CHALLENGES/2023a_SSH_mapping_OSE/sad.tar.gz
Résolution de ige-meom-opendap.univ-grenoble-alpes.fr (ige-meom-opendap.univ-grenoble-alpes.fr)… 129.88.191.39
Connexion à ige-meom-opendap.univ-grenoble-alpes.fr (ige-meom-opendap.univ-grenoble-alpes.fr)|129.88.191.39|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 200
Taille : 482816000 (460M) [application/octet-stream]
Enregistre : ‘sad.tar.gz’

sad.tar.gz          100%[===================>] 460,45M  7,64MB/s    ds 71s     

2025-03-07 14:46:56 (6,46 MB/s) - ‘sad.tar.gz’ enregistré [482816000/482816000]

sad/
sad/distance_to_nearest_coastline_60.nc
sad/land_water_mask_60.nc
sad/variance_cmems_dt_allsat.nc


<div class="alert alert-info" role="alert">

<h2>3. Data for comparison</h2>

</div>

## Download all available maps for comparison

(To date DUACS, MIOST, MIOST-baro-eqwaves and GLORYS12v1 are available)

In [12]:
if os.path.isdir('../data/maps'):
    print('maps data already exist locally in ../data/')
else: 
    print('Downloading maps data')
    
    # Download the alongtrack repository
    !wget https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/MEOM/OCEAN_DATA_CHALLENGES/2023a_SSH_mapping_OSE/maps.tar.gz
   
    # Uncompress in ../data
    !tar -xvf maps.tar.gz --directory ../data
    
    # Remove compressed file
    !rm -f maps.tar.gz 
    

--2025-03-07 14:51:55--  https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/MEOM/OCEAN_DATA_CHALLENGES/2023a_SSH_mapping_OSE/maps.tar.gz
Résolution de ige-meom-opendap.univ-grenoble-alpes.fr (ige-meom-opendap.univ-grenoble-alpes.fr)… 129.88.191.39
Connexion à ige-meom-opendap.univ-grenoble-alpes.fr (ige-meom-opendap.univ-grenoble-alpes.fr)|129.88.191.39|:443… connecté.
requête HTTP transmise, en attente de la réponse… 404 404
2025-03-07 14:51:55 erreur 404 : 404.

tar: maps.tar.gz : open impossible: Aucun fichier ou dossier de ce nom
tar: Error is not recoverable: exiting now


## Read DUACS maps for comparison

(MIOST, MIOST-baro-eqwaves and GLORYS12v1 are similarly read)

In [13]:
%%time
list_of_maps = sorted(glob('../data/maps/DUACS_global_allsat-alg/*.nc'))
ds_maps = xr.open_mfdataset(list_of_maps, combine='nested', concat_dim='time')
ds_maps = ds_maps.sel(time=slice(time_min, time_max))
ds_maps

CPU times: user 25.3 s, sys: 3.58 s, total: 28.9 s
Wall time: 54.9 s


<xarray.Dataset>
Dimensions:    (latitude: 720, longitude: 1440, time: 365)
Coordinates:
  * latitude   (latitude) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * longitude  (longitude) float64 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9
  * time       (time) datetime64[ns] 2019-01-01 2019-01-02 ... 2019-12-31
Data variables:
    sla        (time, latitude, longitude) float64 dask.array<chunksize=(1, 50, 50), meta=np.ndarray>
    ugosa      (time, latitude, longitude) float64 dask.array<chunksize=(1, 50, 50), meta=np.ndarray>
    vgosa      (time, latitude, longitude) float64 dask.array<chunksize=(1, 50, 50), meta=np.ndarray>
    adt        (time, latitude, longitude) float64 dask.array<chunksize=(1, 50, 50), meta=np.ndarray>
    ugos       (time, latitude, longitude) float64 dask.array<chunksize=(1, 50, 50), meta=np.ndarray>
    vgos       (time, latitude, longitude) float64 dask.array<chunksize=(1, 50, 50), meta=np.ndarray>